In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import torch
import torch.nn.functional as F
from torch import nn
import safetensors
from accelerate import notebook_launcher

import einops

from utils.data_utils import BrainDataset, get_tokenizer
from utils.train_utils import TrainConfig, run_train_model, count_parameters, simple_train_model

from transformers import GPT2Tokenizer
from vector_quantize_pytorch import FSQ

import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from simple_parsing import Serializable


In [2]:
from models.vq_brain_per_channel import SoundStream, VAEConfig

In [3]:
model_config = VAEConfig()
print(model_config)

model = SoundStream(**model_config.to_dict())
count_parameters(model)

x = torch.zeros(2, 512, 256)
loss, pred = model(x)

VAEConfig(C=128, n_features=1, levels=(8, 5, 5, 5), stride_list=(2, 2, 2))
self.codebook_size 1000
self.downsample 8
Total: 3.36M, Trainable: 3.36M


### Run training pipeline

In [4]:
window_size = 256
n_electrodes = 256

train_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=4, value=0, always_apply=True),
    A.RandomCrop(height=window_size, width=n_electrodes, always_apply=True),
])

test_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=4, value=0, always_apply=True),
    A.CenterCrop(height=window_size, width=n_electrodes, always_apply=True),
])

In [5]:
# data_path = Path("/drive/data/competitionData")
# test_dataset = BrainDataset(data_path / 'competitionHoldOut', tokenize_function=None, transform=test_transform)
# # x = test_dataset[0][0]
# # plt.imshow(x.T) 


In [6]:
data_path = Path("/drive/data/competitionData")

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_dataset = BrainDataset(data_path / 'train', tokenize_function=get_tokenizer(tokenizer), transform=train_transform)
test_dataset = BrainDataset(data_path / 'test', tokenize_function=get_tokenizer(tokenizer), transform=test_transform)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Runed processing of the  /drive/data/competitionData/train
len of the dataset: 8800
max input len 906
median len 297.0
Runed processing of the  /drive/data/competitionData/test
len of the dataset: 880
max input len 919
median len 283.5


In [ ]:
project_name = 'vq_brain'
save_folder = Path("/drive/logs/kovalev")

train_config = TrainConfig(exp_name='random_sampling_8x_downsample',
                           mixed_precision=True, 
                           batch_size=8, 
                           num_workers=3, 
                           pin_memory=True, 
                           eval_interval=1000, 
                           learning_rate=1e-4,
                           weight_decay=0, 
                           grad_clip=10, 
                           lr_decay_iters=100_000, 
                           warmup_iters=2000, 
                           max_steps = 200_000, 
                           project_name=project_name, 
                           save_folder=save_folder
                          )

model = torch.compile(model)
args = (model, (train_dataset, test_dataset), train_config, model_config)
notebook_launcher(run_train_model, args, num_processes=1)

Launching training on one GPU.


/opt/conda/envs/pytorch/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


Device for training:  cuda
Num devices:  1
Completed initialization of scheduler
**********************************************************************************************

In [ ]:

model = model._orig_mod

In [ ]:
model 

In [ ]:
x = test_dataset[500][0]

x = torch.from_numpy(x[None, :]).to('cuda')

# e = model.encoder(x)
indices, quantized = model.get_quantize_vectors(x)



y = model(x)[1]


print(F.l1_loss(y, x))
print(F.mse_loss(y, x))

x = x.detach().cpu()[0]
y = y.detach().cpu()[0]

import matplotlib.pyplot as plt
plt.plot(y[:, 220], label='recon')
# plt.show()
plt.plot(x[:, 220])
plt.legend()
plt.show()
plt.imshow(y[:, :].T, aspect='auto', vmin=0, vmax=1)
plt.show()
plt.imshow(x[:, :].T, aspect='auto', vmin=0, vmax=1)

In [ ]:
quantized = quantized.detach().cpu()
indices = indices.detach().cpu()
print(quantized.shape)
plt.imshow(quantized[64].T)

plt.show()
plt.imshow(F.one_hot(indices[64].to(torch.long), 1000).T, aspect='auto')
plt.show()
plt.plot(indices[64])

In [ ]:
plt.imshow(quantized[0][:64].T.cpu().detach(), aspect='auto')

In [ ]:
plt.plot(indices[0].cpu())

In [ ]:
quantized.shape

In [ ]:
len(torch.unique(indices))

In [ ]:
indices.shape